In [ ]:
using JLD2
using Statistics
using Printf
using Laplacians

## This notebook outputs the summary tables

In [ ]:
# SuiteSparse 
@load "suitesparse_all.jl.split02merge02.jld2"
dic_suitesparse = copy(dic);

In [ ]:
@load "uniform_grid_all.jl.split02merge02.jld2"
dic1 = copy(dic);

In [6]:
# unweighted chimera
@load "uni_chimera_all.jl.split02merge02.n10000.h1.0.jld2"
dic_uniChimera = copy(dic);
@load "uni_chimera_all.jl.split02merge02.n100000.h2.0.jld2"
dic_uniChimera = merge(combine, dic_uniChimera, dic);
@load "uni_chimera_all.jl.split02merge02.n1000000.h4.0.jld2"
dic_uniChimera = merge(combine, dic_uniChimera, dic);
@load "uni_chimera_all.jl.split02merge02.n10000000.h8.0.jld2"
dic_uniChimera = merge(combine, dic_uniChimera, dic);

In [7]:
# weighted chimera
@load "wted_chimera_nopetsc.jl.split02merge02.n10000.h1.0.jld2"
dic_wtedChimera = copy(dic);
@load "wted_chimera_nopetsc.jl.split02merge02.n100000.h2.0.jld2"
dic_wtedChimera = merge(combine, dic_wtedChimera, dic);
@load "wted_chimera_nopetsc.jl.split02merge02.n1000000.h4.0.jld2"
dic_wtedChimera = merge(combine, dic_wtedChimera, dic);
@load "wted_chimera_nopetsc.jl.split02merge02.n10000000.h8.0.jld2"
dic_wtedChimera = merge(combine, dic_wtedChimera, dic);

In [8]:
# unweighted boundary chimera
@load "uni_bndry_chimera_all.jl.split02merge02.n10000.h1.0.jld2"
dic_uniBndryChimera = copy(dic);
@load "uni_bndry_chimera_all.jl.split02merge02.n100000.h2.0.jld2"
dic_uniBndryChimera = merge(combine, dic_uniBndryChimera, dic);
@load "uni_bndry_chimera_all.jl.split02merge02.n1000000.h4.0.jld2"
dic_uniBndryChimera = merge(combine, dic_uniBndryChimera, dic);
@load "uni_bndry_chimera_all.jl.split02merge02.n10000000.h8.0.jld2"
dic_uniBndryChimera = merge(combine, dic_uniBndryChimera, dic);

In [9]:
# weighted boundary chimera
@load "wted_bndry_chimera_nopetsc.jl.split02merge02.n10000.h1.0.jld2"
dic_wtedBndryChimera = copy(dic)
@load "wted_bndry_chimera_nopetsc.jl.split02merge02.n100000.h2.0.jld2"
dic_wtedBndryChimera = merge(combine, dic_wtedBndryChimera, dic);
@load "wted_bndry_chimera_nopetsc.jl.split02merge02.n1000000.h4.0.jld2"
dic_wtedBndryChimera = merge(combine, dic_wtedBndryChimera, dic);
@load "wted_bndry_chimera_nopetsc.jl.split02merge02.n10000000.h8.0.jld2"
dic_wtedBndryChimera = merge(combine, dic_wtedBndryChimera, dic);

In [10]:
# max flow 
@load "chimeraIPM_nopetsc.jl.split02merge02.jld2"
dic_maxFlow = copy(dic);
@load "spielmanIPM_all.jl.split02merge02.jld2"
dic_maxFlow = merge(combine, dic_maxFlow, dic);

In [11]:
# Sachdeva star 
@load "sachdeva_star_all.jl.split02merge02.jld2"
dic_sachdevaStar = copy(dic);

In [12]:
# SPE 
@load "spe_all.jl.split02merge02.jld2"
dic_spe = copy(dic);

In [13]:
# uniform grid
@load "uniform_grid_all.jl.split02merge02.jld2"
dic_uniGrid = copy(dic);

In [14]:
# checkered grid (high contrast grid)
@load "checkered_all.jl.split02merge02.jld2"
dic_checkeredGrid = copy(dic);

In [15]:
# aniso grid 
@load "aniso_all.jl.split02merge02.jld2"
dic_anisoGrid = copy(dic);

In [16]:
# weighted grid 
@load "wgrid_all.jl.split02merge02.jld2"
dic_wtedGrid = copy(dic);

### done

In [22]:
dicAll = [dic_suitesparse, dic_uniChimera, dic_wtedChimera, dic_uniBndryChimera, dic_wtedBndryChimera, dic_maxFlow, dic_sachdevaStar, dic_spe, dic_uniGrid, dic_checkeredGrid,dic_anisoGrid,dic_wtedGrid]

In [23]:
instanceNames = ["SuiteSparse Matrix Collection",  "Unweighted chimeras", "Weighted chimeras", "Unweighted SDDM chimeras", "Weighted SDDM chimeras", "Maximum flow IPMs", "Barbell stars", "SPE benchmark", "Uniform coefficient Poisson grid", "High contrast coefficient Poisson grid", "Anisotropic coef. Poisson grid, variable discretization", "Anisotropic coef. Poisson grid, variable weight"];

In [24]:
function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=3)
end        
ff(s) = (@sprintf "%.4g" sd(3,s))
nf(s) = (@sprintf "%d" sd(3,s))
ffK(s) = (@sprintf "%.4g" sd(3,s)/1000)
nfK(s) =  round(Int64, sd(3,s)/1000)
ffe8(s) = (@sprintf "%.3g" sd(3,s)*1e8)
ffe6(s) = (@sprintf "%.3g" sd(3,s)*1e6)

ffe6 (generic function with 1 method)

In [25]:
solvers=["ac", "ac-s2m2", "cmg", "hypre", "petsc_hypre",  "icc"];

In [26]:
function get_tvn_max(d, solvers)
    
    results_array = []
    for solver in solvers 
        if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
            cur_err = maximum(d[string(solver, "_err")])
            tvn = maximum((d[string(solver, "_tot")] ./ d["ne"]))
            if cur_err <= 1e-8
                push!(results_array, (solver, ffe6(tvn)))
            elseif cur_err < 1e-4
                push!(results_array, (solver, string(ffe6(tvn), "\\tnote{*}")))
            elseif cur_err < 1
                push!(results_array, (solver, string(ffe6(tvn), "\\tnote{**}")))
            else
                push!(results_array, (solver, "Inf"))
            end
        else
            push!(results_array, (solver, "N/A"))
        end
    end

    results = Dict(results_array)
    return results
end

get_tvn_max (generic function with 1 method)

In [27]:
for (index, d) in enumerate(dicAll)
    max_tvns = get_tvn_max(d, solvers)
    line = string("{", instanceNames[index], "}&", join([max_tvns[solver] for solver in solvers], "&"), "\\\\")
    println(line)
end

{SuiteSparse Matrix Collection}&0.994&1.4&Inf&Inf&27.1&153\\
{Unweighted chimeras}&3.95&7.19&Inf&Inf&Inf&Inf\\
{Weighted chimeras}&4.39&4.79&Inf&Inf&N/A&Inf\\
{Unweighted SDDM chimeras}&3.4&5.44&Inf&Inf&Inf&Inf\\
{Weighted SDDM chimeras}&4.34&4.36&Inf&Inf&N/A&Inf\\
{Maximum flow IPMs}&1.2&2.39&1.9\tnote{**}&5.04\tnote{*}&N/A&Inf\\
{Barbell stars}&4.66\tnote{**}&1.05&0.868&3.52\tnote{**}&7.22\tnote{**}&Inf\\
{SPE benchmark}&1.62&1.91&0.511&0.648&2.58&4.29\\
{Uniform coefficient Poisson grid}&1.46&2.49&0.624&0.587&2.62&2.22\\
{High contrast coefficient Poisson grid}&2.33&3.57&0.572&0.752&3.85&4.76\\
{Anisotropic coef. Poisson grid, variable discretization}&1.48&2.5&0.659&0.614&2.6&1.92\\
{Anisotropic coef. Poisson grid, variable weight}&1.67&2.5&0.674&0.772&4.92&2.44\\
